In [ ]:
!pip install -q transformers datasets scikit-learn

import pandas as pd
import numpy as np
import torch
import os
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset, load_dataset

# Set Device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

In [ ]:
# ==========================================
# 1. LOAD UNSAFE DATA (Class 1)
# ==========================================
print("Loading UNSAFE data (Big Porn Dataset)...")

# We only load 40,000 rows. This is enough to learn patterns without crashing RAM.
# We use the special '‽' separator you found.
unsafe_df = pd.read_csv(
    "/kaggle/input/big-porn-dataset-fyp/data.csv", 
    sep='‽', 
    engine='python', 
    on_bad_lines='skip',
    nrows=40000 
)

# Combine Title and Tags into one text string for the model
unsafe_df['text'] = unsafe_df['title'].fillna("") + " " + unsafe_df['tags'].fillna("")
unsafe_df['label'] = 1  # 1 = UNSAFE
unsafe_df = unsafe_df[['text', 'label']]

print(f"✅ Loaded {len(unsafe_df)} Unsafe samples.")

# ==========================================
# 2. LOAD SAFE DATA (Class 0)
# ==========================================
print("Loading SAFE data (WikiText)...")
# We use Wikipedia as a proxy for "Safe/General" internet content
safe_dataset = load_dataset("wikitext", "wikitext-2-v1", split="train[:40000]")

safe_list = [x['text'] for x in safe_dataset if len(x['text']) > 50] # Filter short junk
safe_df = pd.DataFrame({'text': safe_list, 'label': 0}) # 0 = SAFE
safe_df = safe_df.head(40000) # Match size of Unsafe data

print(f"✅ Loaded {len(safe_df)} Safe samples.")

# ==========================================
# 3. LOAD MEDICAL CONTEXT (Class 2)
# ==========================================
print("Loading MEDICAL data (AG News)...")
# This ensures we don't block biology/health sites
sci_dataset = load_dataset("ag_news", split="train[:50000]")

medical_keywords = ['health', 'cancer', 'treatment', 'patient', 'doctor', 'surgery', 'anatomy', 'biology', 'reproductive', 'breast', 'virus']
medical_list = []

for item in sci_dataset:
    if item['label'] == 3: # 3 is Sci/Tech
        if any(word in item['text'].lower() for word in medical_keywords):
            medical_list.append(item['text'])

# Augment (Duplicate) medical data if it's too small, so the model learns it well
medical_df = pd.DataFrame({'text': medical_list, 'label': 2}) # 2 = MEDICAL
if len(medical_df) < 5000:
    medical_df = pd.concat([medical_df] * 5, ignore_index=True)
medical_df = medical_df.head(40000)

print(f"✅ Loaded {len(medical_df)} Medical samples.")

# ==========================================
# 4. MERGE EVERYTHING
# ==========================================
df = pd.concat([unsafe_df, safe_df, medical_df], ignore_index=True)
df = df.sample(frac=1, random_state=42).reset_index(drop=True) # Shuffle

print("\nFinal Dataset Distribution:")
print(df['label'].value_counts())

In [ ]:
# 1. SPLIT TRAIN / TEST
train_df, val_df = train_test_split(df, test_size=0.15, random_state=42)

# Convert to Hugging Face format
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

# 2. TOKENIZER
model_name = "distilbert-base-uncased"
tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)

def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=128)

print("Tokenizing... (This takes 1-2 mins)")
train_tokenized = train_dataset.map(tokenize, batched=True)
val_tokenized = val_dataset.map(tokenize, batched=True)

# 3. LOAD MODEL
model = DistilBertForSequenceClassification.from_pretrained(
    model_name,
    num_labels=3,
    id2label={0: "SAFE", 1: "UNSAFE", 2: "MEDICAL"},
    label2id={"SAFE": 0, "UNSAFE": 1, "MEDICAL": 2}
).to(device)

# 4. TRAINING CONFIG (Corrected for New Version)
training_args = TrainingArguments(
    output_dir="./fyp_model_output",
    num_train_epochs=2,              
    per_device_train_batch_size=32,  
    per_device_eval_batch_size=64,
    eval_strategy="epoch",           # <--- CHANGED FROM evaluation_strategy
    save_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
)

# 5. RUN TRAINING
print("🚀 Starting Training...")
trainer.train()

In [ ]:
# 1. SAVE TO DISK
save_path = "fyp_final_model"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)
print(f"Model saved to {save_path}")

# 2. TEST IT IMMEDIATELY
def predict(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=128).to(device)
    with torch.no_grad():
        logits = model(**inputs).logits
    pred_id = logits.argmax().item()
    return model.config.id2label[pred_id]

print("\n--- TEST RESULTS ---")
print("Test 1 (Safe):", predict("The python programming language is great."))
print("Test 2 (Porn):", predict("Hardcore xxx video free download."))
print("Test 3 (Medical):", predict("The anatomy of the reproductive system."))

# 3. ZIP FOR DOWNLOAD
!zip -r fyp_final_model.zip fyp_final_model
print("\n✅ DONE! Download 'fyp_final_model.zip' from the Output tab on the right.")

In [ ]:
### Rigirous testiing
import pandas as pd
import torch
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
from datasets import load_dataset
from tqdm import tqdm

# ==========================================
# 1. SETUP
# ==========================================
device = "cuda" if torch.cuda.is_available() else "cpu"
MODEL_PATH = "fyp_final_model" # Folder where you saved the model

print(f"Loading model from {MODEL_PATH} on {device}...")
try:
    tokenizer = DistilBertTokenizerFast.from_pretrained(MODEL_PATH)
    model = DistilBertForSequenceClassification.from_pretrained(MODEL_PATH).to(device)
except OSError:
    print("❌ Model not found! Did you run the training cell?")
    exit()

# ==========================================
# 2. CREATE "HARD MODE" TEST DATASET
# ==========================================
print("\nBUILDING RIGOROUS TEST SET...")

# A. UNSEEN PORN (Skip the first 50k rows used for training)
print("1. Loading Unseen Porn (Rows 50k-51k)...")
df_porn = pd.read_csv(
    "/kaggle/input/big-porn-dataset-fyp/data.csv", 
    sep='‽', engine='python', on_bad_lines='skip',
    skiprows=50000, nrows=500  # <--- SKIPPING TRAINING DATA
)
df_porn['text'] = df_porn.iloc[:, 2].fillna("") + " " + df_porn.iloc[:, 3].fillna("") # Title + Tags
df_porn['true_label'] = 1 # UNSAFE
df_porn = df_porn[['text', 'true_label']]

# B. UNSEEN MEDICAL (PubMed Abstracts - Highly Technical)
print("2. Loading Unseen Medical (PubMed)...")
ds_med = load_dataset("pubmed_qa", "pqa_labeled", split="train[:500]")
df_med = pd.DataFrame({'text': ds_med['question'], 'true_label': 2}) # 2 = MEDICAL

# C. UNSEEN SAFE (IMDB Movie Reviews - Casual/Slang)
print("3. Loading Unseen Safe (IMDB Reviews)...")
ds_safe = load_dataset("imdb", split="test[:500]")
df_safe = pd.DataFrame({'text': ds_safe['text'], 'true_label': 0}) # 0 = SAFE

# Combine and Shuffle
test_df = pd.concat([df_porn, df_med, df_safe], ignore_index=True)
test_df = test_df.sample(frac=1, random_state=42).reset_index(drop=True)

print(f"✅ Created Test Set with {len(test_df)} distinct samples.")

# ==========================================
# 3. EVALUATION LOOP
# ==========================================
print("\nRunning Evaluation (This takes ~30 seconds)...")

predictions = []
true_labels = test_df['true_label'].tolist()

# Run in batches to be fast
batch_size = 32
texts = test_df['text'].tolist()

model.eval() # Set model to evaluation mode

for i in tqdm(range(0, len(texts), batch_size)):
    batch_texts = texts[i : i + batch_size]
    
    # Tokenize
    inputs = tokenizer(
        batch_texts, 
        return_tensors="pt", 
        truncation=True, 
        padding=True, 
        max_length=128
    ).to(device)
    
    # Predict
    with torch.no_grad():
        logits = model(**inputs).logits
    
    # Store predictions
    preds = torch.argmax(logits, dim=1).cpu().numpy()
    predictions.extend(preds)

# ==========================================
# 4. METRICS & VISUALIZATION
# ==========================================
target_names = ["SAFE", "UNSAFE (Porn)", "MEDICAL"]

print("\n" + "="*30)
print("   CLASSIFICATION REPORT")
print("="*30)
print(classification_report(true_labels, predictions, target_names=target_names))

# Plot Confusion Matrix
cm = confusion_matrix(true_labels, predictions)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=target_names, yticklabels=target_names)
plt.xlabel('Predicted by Model')
plt.ylabel('Actual Label')
plt.title('Where did the model get confused?')
plt.show()

In [ ]:
# ==========================================
# REPAIR & FINE-TUNE SCRIPT (WITH .PTH EXPORT)
# ==========================================
import os
import torch
from datasets import load_dataset, Dataset
import pandas as pd
from transformers import Trainer, TrainingArguments

# 1. DISABLE WANDB
os.environ["WANDB_DISABLED"] = "true"

print("Loading conversational data for fine-tuning...")

# Unsafe (Comments)
ds_unsafe = load_dataset("google/civil_comments", split="train[:20000]")
unsafe_list = [x['text'] for x in ds_unsafe if x['sexual_explicit'] > 0.5][:1000]
df_unsafe = pd.DataFrame({'text': unsafe_list, 'label': 1})

# Safe (News)
ds_safe = load_dataset("cnn_dailymail", "3.0.0", split="train[:1000]")
safe_list = [x['article'][:500] for x in ds_safe]
df_safe = pd.DataFrame({'text': safe_list, 'label': 0})

# Medical (Drugs)
ds_med = load_dataset("ade_corpus_v2", "Ade_corpus_v2_classification", split="train[:1000]")
med_list = [x['text'] for x in ds_med]
df_med = pd.DataFrame({'text': med_list, 'label': 2})

# Merge dataset
repair_df = pd.concat([df_unsafe, df_safe, df_med], ignore_index=True)
repair_df = repair_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Convert to HF Dataset
repair_dataset = Dataset.from_pandas(repair_df)

# Tokenize
repair_tokenized = repair_dataset.map(
    lambda x: tokenizer(
        x["text"], 
        padding="max_length",
        truncation=True,
        max_length=128
    ), 
    batched=True
)

print(f"Fine-tuning on {len(repair_df)} new conversational samples...")

# 3. TRAINER
training_args = TrainingArguments(
    output_dir="./fyp_repaired_model",
    num_train_epochs=2,
    per_device_train_batch_size=16,
    learning_rate=1e-5,
    save_strategy="no",
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=repair_tokenized,
)

print("Starting Fine-Tuning...")
trainer.train()

# 4. SAVE HUGGINGFACE MODEL
save_dir = "fyp_final_model_v2"
model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)

# 5. SAVE AS .PTH CHECKPOINT
pth_path = "fyp_final_model_v2/model.pth"
torch.save(model.state_dict(), pth_path)

print("Saved HF model at:", save_dir)
print("Saved PyTorch .pth file at:", pth_path)
print("Done.")

In [ ]:
import torch
import pandas as pd
from datasets import load_dataset
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
from sklearn.metrics import classification_report
from tqdm import tqdm

# ==========================================
# 1. SETUP - POINT TO THE NEW v2 MODEL
# ==========================================
device = "cuda" if torch.cuda.is_available() else "cpu"
MODEL_PATH = "fyp_final_model_v2"  # <--- USING THE REPAIRED BRAIN

print(f"Loading Improved Model from {MODEL_PATH}...")
try:
    tokenizer = DistilBertTokenizerFast.from_pretrained(MODEL_PATH)
    model = DistilBertForSequenceClassification.from_pretrained(MODEL_PATH).to(device)
except OSError:
    print("❌ Error: Model not found. Did the fine-tuning finish?")
    exit()

# ==========================================
# 2. PREPARE TEST DATA (Unseen Samples)
# ==========================================
print("\nLoading test data...")

# A. UNSAFE (Comments) - We test on rows 50,000+ (Unseen)
ds_unsafe = load_dataset("google/civil_comments", split="train[50000:52000]") # Different chunk than training
unsafe_list = [x['text'] for x in ds_unsafe if x['sexual_explicit'] > 0.5][:300]
df_unsafe = pd.DataFrame({'text': unsafe_list, 'true_label': 1})

# B. SAFE (News) - Test split
ds_safe = load_dataset("cnn_dailymail", "3.0.0", split="test[:300]")
safe_list = [x['article'][:500] for x in ds_safe]
df_safe = pd.DataFrame({'text': safe_list, 'true_label': 0})

# C. MEDICAL (Drugs) - Test split (if available) or end of train
ds_med = load_dataset("ade_corpus_v2", "Ade_corpus_v2_classification", split="train[-300:]") # Last 300
med_list = [x['text'] for x in ds_med]
df_med = pd.DataFrame({'text': med_list, 'true_label': 2})

# Combine
test_df = pd.concat([df_unsafe, df_safe, df_med], ignore_index=True)
test_df = test_df.sample(frac=1, random_state=42).reset_index(drop=True)
print(f"✅ Ready to test on {len(test_df)} samples.")

# ==========================================
# 3. RUN PREDICTION
# ==========================================
predictions = []
true_labels = test_df['true_label'].tolist()
texts = test_df['text'].tolist()

model.eval()
batch_size = 32

for i in tqdm(range(0, len(texts), batch_size)):
    batch_texts = texts[i : i + batch_size]
    inputs = tokenizer(batch_texts, return_tensors="pt", truncation=True, padding=True, max_length=128).to(device)
    with torch.no_grad():
        logits = model(**inputs).logits
    predictions.extend(torch.argmax(logits, dim=1).cpu().numpy())

# ==========================================
# 4. RESULTS
# ==========================================
target_names = ["SAFE (News)", "UNSAFE (Comments)", "MEDICAL (Drugs)"]
print("\n" + "="*40)
print("   IMPROVED MODEL RESULTS (v2)")
print("="*40)
print(classification_report(true_labels, predictions, target_names=target_names))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report

# Class labels
class_names = ["SAFE (News)", "UNSAFE (Comments)", "MEDICAL (Drugs)"]
labels = [0, 1, 2]

# ===============================
# 1. CONFUSION MATRIX
# ===============================
cm = confusion_matrix(true_labels, predictions, labels=labels)

disp = ConfusionMatrixDisplay(
    confusion_matrix=cm,
    display_labels=class_names
)

plt.figure()
disp.plot(cmap=None)
plt.title("Confusion Matrix for Content Classification Model v2")
plt.tight_layout()
plt.show()

# ===============================
# 2. F1 SCORE BAR GRAPH
# ===============================
report = classification_report(
    true_labels,
    predictions,
    target_names=class_names,
    output_dict=True
)

f1_scores = [
    report["SAFE (News)"]["f1-score"],
    report["UNSAFE (Comments)"]["f1-score"],
    report["MEDICAL (Drugs)"]["f1-score"]
]

plt.figure()
plt.bar(class_names, f1_scores)
plt.ylim(0, 1.05)
plt.ylabel("F1 Score")
plt.title("F1 Score per Class for Model v2")

for i, score in enumerate(f1_scores):
    plt.text(i, score + 0.01, f"{score:.2f}", ha="center")

plt.tight_layout()
plt.show()

In [ ]:
import torch
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datasets import load_dataset
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
from sklearn.metrics import classification_report, confusion_matrix
from tqdm import tqdm

# ==========================================
# 1. SETUP
# ==========================================
device = "cuda" if torch.cuda.is_available() else "cpu"
MODEL_PATH = "fyp_final_model_v2" # Using the REPAIRED model

print(f"Loading Fine-Tuned Brain from {MODEL_PATH}...")
try:
    tokenizer = DistilBertTokenizerFast.from_pretrained(MODEL_PATH)
    model = DistilBertForSequenceClassification.from_pretrained(MODEL_PATH).to(device)
except OSError:
    print("❌ Error: Model not found. Please run the Fine-Tuning cell first!")
    exit()

# ==========================================
# 2. LOAD 3 NEW DATASETS (The Rigorous Test)
# ==========================================
print("\n📥 Downloading 3 NEW datasets for Stress Testing...")

# --- A. UNSAFE TEST (Offensive Tweets) ---
# Dataset: tweet_eval (offensive)
# Challenge: Slang, typos, short text.
print("1. Loading 'TweetEval' (Offensive Tweets)...")
ds_unsafe = load_dataset("tweet_eval", "offensive", split="test")
# Filter for label 1 (Offensive) and take 300
unsafe_list = [x['text'] for x in ds_unsafe if x['label'] == 1][:300]
df_unsafe = pd.DataFrame({'text': unsafe_list, 'true_label': 1}) # 1 = UNSAFE
print(f"   -> Found {len(df_unsafe)} offensive tweets.")

# --- B. SAFE TEST (General Knowledge) ---
# Dataset: dbpedia_14
# Challenge: Encyclopedic text about Companies/Artists (Must be allowed)
print("2. Loading 'DBPedia' (General Knowledge)...")
ds_safe = load_dataset("dbpedia_14", split="test[:300]")
safe_list = [x['content'][:200] for x in ds_safe] # First 200 chars
df_safe = pd.DataFrame({'text': safe_list, 'true_label': 0}) # 0 = SAFE
print(f"   -> Found {len(df_safe)} encyclopedia entries.")

# --- C. MEDICAL TEST (Exam Questions) ---
# Dataset: medmcqa
# Challenge: Very complex biology/anatomy questions.
print("3. Loading 'MedMCQA' (Medical Exams)...")
ds_med = load_dataset("medmcqa", split="test[:300]")
med_list = [x['question'] for x in ds_med]
df_med = pd.DataFrame({'text': med_list, 'true_label': 2}) # 2 = MEDICAL
print(f"   -> Found {len(df_med)} medical questions.")

# Combine
test_df = pd.concat([df_unsafe, df_safe, df_med], ignore_index=True)
test_df = test_df.sample(frac=1, random_state=42).reset_index(drop=True)

print(f"\n✅ Ready to test on {len(test_df)} completely new samples.")

# ==========================================
# 3. RUN PREDICTIONS
# ==========================================
predictions = []
true_labels = test_df['true_label'].tolist()
texts = test_df['text'].tolist()

batch_size = 32
model.eval()

print("Running AI analysis...")
for i in tqdm(range(0, len(texts), batch_size)):
    batch_texts = texts[i : i + batch_size]
    
    inputs = tokenizer(
        batch_texts, return_tensors="pt", truncation=True, padding=True, max_length=128
    ).to(device)
    
    with torch.no_grad():
        logits = model(**inputs).logits
    
    preds = torch.argmax(logits, dim=1).cpu().numpy()
    predictions.extend(preds)

# ==========================================
# 4. FINAL REPORT & VISUALIZATION
# ==========================================
target_names = ["SAFE (DBPedia)", "UNSAFE (Tweets)", "MEDICAL (Exams)"]

print("\n" + "="*40)
print("   FINAL RIGOROUS EVALUATION REPORT")
print("="*40)
print(classification_report(true_labels, predictions, target_names=target_names))

# Create Confusion Matrix
cm = confusion_matrix(true_labels, predictions)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Greens', xticklabels=target_names, yticklabels=target_names)
plt.xlabel('AI Predicted')
plt.ylabel('Actual Category')
plt.title('Final Model Robustness Check')
plt.show()

# Show Mistakes (If any)
print("\n--- ANALYZING MISTAKES ---")
mistakes = 0
for i in range(len(texts)):
    if predictions[i] != true_labels[i]:
        print(f"Mistake: AI said '{target_names[predictions[i]]}' | Real is '{target_names[true_labels[i]]}'")
        print(f"Text: {texts[i][:100]}...\n")
        mistakes += 1
        if mistakes > 5: break

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc

# ===============================
# 1. GET PROBABILITIES
# ===============================
model.eval()
all_probs = []

batch_size = 32
for i in range(0, len(texts), batch_size):
    batch_texts = texts[i : i + batch_size]
    inputs = tokenizer(
        batch_texts,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=128
    ).to(device)

    with torch.no_grad():
        logits = model(**inputs).logits
        probs = torch.softmax(logits, dim=1)

    all_probs.append(probs.cpu().numpy())

y_score = np.vstack(all_probs)
y_true = np.array(true_labels)

# ===============================
# 2. BINARIZE LABELS
# ===============================
class_names = ["SAFE (News)", "UNSAFE (Comments)", "MEDICAL (Drugs)"]
y_true_bin = label_binarize(y_true, classes=[0, 1, 2])

# ===============================
# 3. ROC CURVES
# ===============================
plt.figure()

for i, class_name in enumerate(class_names):
    fpr, tpr, _ = roc_curve(y_true_bin[:, i], y_score[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f"{class_name} (AUC = {roc_auc:.2f})")

plt.plot([0, 1], [0, 1], linestyle="dotted")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curves for Content Classification Model v2")
plt.legend()
plt.tight_layout()
plt.show()